<a href="https://colab.research.google.com/github/R0bk/ml_replications/blob/main/notes/Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Score

A score is the graident of the probability density w.r.t. the input dimensions (not w.r.t. model parameters of a model trying to replciate the prob density)


Why work with scores?
One issue with PDFs is that they are normalised, as in the integral has always to be 1, but for scores this doesn't have to be true so instead they can be an arbitrary function.

##EBMs
An example of using this would be with a deep energy based model (EBM).

Since we want a flexiable model that can map any function we can use a deep neural network (DNN) that takes a sample $x$ and maps it to a real number

$f_θ(x)\inℝ$

Since we know the probability density has to be non-negitive we can take the exponential of the negativive of the pd. But to be a valid PDF we need to ensure that the $\int_a^b p_\theta(x) dx=1$. So to do this we need to divide by a normalizing constant, this gives us the model.

$p_θ(x)=\frac{e^{-f_\theta(x)}}{Z_θ}$

We can plug in any neural network we want into this model to find the PDF but the problem is that the normalizer $Z_\theta$ may be intractable or very hard to calculate. You can see this if we try to learn $\theta$ by maximum likelihood.

$𝔼_{p_{data}}[-log\;p_\theta(x)] = 𝔼_{p_{data}}[log\;f_\theta(x)-log\;Z_\theta]$


## Score based
If we take the gradient of the MLE function above w.r.t. $x$ we get the below.

$\nabla_xlog\;p_\theta(x)=-\nabla_x\;f_\theta(x)-\nabla_xlog\;Z_\theta$

But we can see that the last term w.r.t. $x$ will actually be 0 and we don't need to comute it giving us a tractable function

$\nabla_xlog\;p_\theta(x)=-\nabla_x\;f_\theta(x)$

So if we can build a model that can attempt to learn in the vector field of graidents instead of in the probability density space we can avoid the intractable computation all togther

##Score Estimation
So how do we learn actually given the data, well given iid samples

$\{x_1, x_2, ... x_N\} \stackrel{i.i.d.}{\sim} p_{data}(x)=p(x)$

And given the task of estimating the score 

$\nabla_xlog\;p_\theta(x)$

And given the score model (a vector valued function)

$s_\theta(x): ℝ^D \to ℝ^D$

We need to figure out a way to compare two vector fields of score, e.g. compare $\nabla_xlog\;p_\theta(x)$ and $s_\theta(x)$

## Score Matching
We can take essentially a l2 norm between the two vector fields as

$\frac{1}{2}𝔼_{p(x)}[||\nabla_xlog\;p(x)-s_\theta(x)||_2^2]$

The l2 norm in this case is called the Fisher divergence

The problem with this approach is that we don't know $\nabla_xlog\;p(x)$ as we don't know $p(x)$

We can apply a trick under the assumption that the graidents disapear on the boundary and do integration by parts to get

$𝔼_{p(x)}[||\frac{1}{2}s_\theta(x)||_2^2 + trace(\;\nabla_xs_\theta(x)\;)]$

Where $\nabla_xs_\theta(x)$ is the Jacobian of $s_\theta(x)$

This is called score matching (full proof is in 2005 paper).

With data we can write this objective as

$\frac{1}{N}\sum_{i=1}^N[||\frac{1}{2}s_\theta(x_i)||_2^2 + trace(\;\nabla_xs_\theta(x_i)\;)]$

for

$\{x_1, x_2, ... x_N\} \stackrel{i.i.d.}{\sim}p(x)$


### Score matching example
We create a DNN that will be used as $s_\theta(x)$. Now we need to calculate  $\frac{1}{N}\sum_{i=1}^N[||\frac{1}{2}s_\theta(x_i)||_2^2 + trace(\;\nabla_xs_\theta(x_i)\;)]$ in order to provide updates to the network. We can do this in two parts

First we calculate $||\frac{1}{2}s_\theta(x)||_2^2$ by simply doing a forward pass of the network and then taking the norm but to calculate $trace(\;\nabla_xs_\theta(x_i)\;)$ we need to first calculate the jacobian of the network.

The jacobian $\nabla_xs_\theta(x_i)=
\begin{bmatrix}
  \frac{\partial s_{\theta,1}(x)}{\partial x_1} & 
    \frac{\partial s_{\theta,1}(x)}{\partial x_2} & 
    \frac{\partial s_{\theta,1}(x)}{\partial x_3} \\[1ex]
  \frac{\partial s_{\theta,2}(x)}{\partial x_1} & 
    \frac{\partial s_{\theta,2}(x)}{\partial x_2} & 
    \frac{\partial s_{\theta,2}(x)}{\partial x_3} \\[1ex]
  \frac{\partial s_{\theta,3}(x)}{\partial x_1} & 
    \frac{\partial s_{\theta,3}(x)}{\partial x_2} & 
    \frac{\partial s_{\theta,3}(x)}{\partial x_3}
\end{bmatrix}$

Requires us to take the deritive of each output of the network $s_{\theta,output}(x)$ with respect to each input of the network. This means that to calculate the jacobian we are required to backprop once for every output in the network. (Note since we take the trace we only consider the diag entries) 

##Sliced score matching
Since calculating the score match requires $O(N)$ parses where n is the number of data dimensions we want to find a way to do it faster. The idea with sliced score matching is to select a random direction and then project the vector field of the model output $\nabla_x\;f_\theta(x)$ and the vector field of the data $\nabla_xlog\;p_\theta(x)$ and project them onto a random $1d$ vector where we can more easily compute the divergence.

This leads to sliced fisher divergence where we project every point $p$ onto a random direction $v$ giving us

$\frac{1}{2}𝔼_{p_v}𝔼_{p_{data}}[(v^T\nabla_xlog\;p_\theta(x)-v^Tlog\;p_{data}(x))^2]$

but once again $p_{data}$ is intractable but just as before if we do the integration by parts trick we get out a divergence that looks like

$𝔼_{p_v}𝔼_{p_{data}}[v^T\nabla_x^2log\;p_\theta(x)v+\frac{1}{2}(v^Tlog\;p_\theta(x))^2] + const$

Now both parts of the comparison of function are fast! :)


###Notes:
When selecting the distribution to sample random directions from $p_v$ the distribution can have a big impact on trainning, ideally you want one that shows contrast between the data and the model gradients.

You can also sample multiple projections per data sample for variance reduction if you want.

##Sampling score matching (Langevin dynamics)
Once we have a trained model that can predict the graident vector field of the data probability density $\nabla_xlog\;p_\theta(x)$ we can now try to sample from it. First we initialise a random sample $\hat{x}_0 \sim \pi(x)$ from any distribution of choice (similar to MCMC) then for step $t$ in range $T$ we can do the following process

$\hat{x}_t \leftarrow \hat{x}_{t-1}+\frac{\epsilon}{2}\nabla_xlog\;p_\theta(\hat{x}_{t-1}) + \sqrt{\epsilon}\;z_t$

where $z_t \sim 𝓝(0,I)$ is added to increase exploration. Essentially we are just choosing a random point and following the graident down with some minor exploration to try and not find local minimas.

##Problems
Projecting a high dimensional manifold onto low dimensional space can result in undefined gradients.

Since we only learn gradients near data points (and lean the best gradients from the most common data points) much of the gradient landscape won't have learnt the true distribution and when attempting to sample the sample can get lost.

Also since we sample from a distribution and follow that sample down the models graident landscape we are likely to end up in whatever minima that is most near by. This doesn't respect that some minimas may be more representitive of a mode in a dataset e.g. a dataset class vs others. So the average of the modes of Langevin dynamic samples will differ from the average of the modes of the true dataset.

##Great Refrences

https://ajolicoeur.wordpress.com/the-new-contender-to-gans-score-matching-with-langevin-sampling/

https://yang-song.github.io/blog/2021/score/

https://www.youtube.com/watch?v=8TcNXi3A5DI